In [6]:
import pandas as pd
raw_data = pd.read_csv("../data/raw/2019-Oct.csv")

ModuleNotFoundError: No module named 'pandas'

In [3]:
raw_data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [4]:
raw_data.isna().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [5]:
percentage_missing_per_column = raw_data.isna().mean() * 100

print(percentage_missing_per_column)

event_time        0.000000
event_type        0.000000
product_id        0.000000
category_id       0.000000
category_code    31.839818
brand            14.410502
price             0.000000
user_id           0.000000
user_session      0.000005
dtype: float64


In [7]:
percentage_rows_with_any_na = (raw_data.isna().any(axis=1).mean()) * 100

print(percentage_rows_with_any_na)

37.42899086531707


In [10]:
# Group by category_id
grouped = raw_data.groupby('category_id')['category_code']

# Check if both NA and non-NA values exist for any group
result = grouped.apply(lambda x: x.isna().any() and x.notna().any()).any()

#Show the results (if any)
print(result)

False


In [ ]:
from torch.utils.data import DataLoader
from scripts.read import NextItemDataset  # Assuming the class is in scripts/read.py

# Create dataset instance
dataset = NextItemDataset(
    path='path/to/your/data.csv',  # Replace with your CSV path
    mode='gru',  # or 'graph'
    max_seq_len=50  # Optional: set maximum sequence length
)

: 

In [1]:
import os

# Print current working directory
print("Current working directory:", os.getcwd())

# List contents of current directory
print("\nContents of current directory:", os.listdir())

# Try to list contents of scripts directory if it exists
scripts_path = os.path.join(os.getcwd(), 'scripts')
if os.path.exists(scripts_path):
    print("\nContents of scripts directory:", os.listdir(scripts_path))
else:
    print("\nscripts directory not found in:", scripts_path)

Current working directory: /Users/miguelpalospou/Desktop/Data_projects/master_project/aidl-upc-w20245-rec-sys/notebooks

Contents of current directory: ['umap_visualization.ipynb', 'data_exploration.ipynb', 'experiments', 'performance_comparison.ipynb']

scripts directory not found in: /Users/miguelpalospou/Desktop/Data_projects/master_project/aidl-upc-w20245-rec-sys/notebooks/scripts


In [2]:
import os
import sys

# Add the project root directory (one level up from notebooks) to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print("Added to Python path:", project_root)

# Now try importing
from scripts.preprocessing_scripts.preprocess_graph_with_encoding import preprocess_graph_with_onehot

# Load config
import yaml
with open('../experiments/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

exp_params = config['experiments']['experiment_7']

# Preprocess data and create datasets
preprocess_graph_with_onehot(
    input_folder_path=exp_params['data_params']['input_folder_path'],
    output_folder_artifacts=exp_params['data_params']['output_folder_artifacts'],
    preprocessing_params=exp_params['preprocessing']
)

Added to Python path: /Users/miguelpalospou/Desktop/Data_projects/master_project/aidl-upc-w20245-rec-sys
Applying preprocessing for Graph NN with embeddings for categorical features...
[INFO] Initializing SessionGraphDataset...
[INFO] Loading CSV files from folder: /Users/miguelpalospou/Desktop/Data_projects/master_project/
[INFO] Loaded 42448764 rows of data from 1 CSV files.
[INFO] Normalizing price column using min_max...
[INFO] Data limited to 40779399 rows with event_type = 'view'.
[INFO] Data limited to 25201706 rows with no nulls in 'brand', 'category_code' and 'price'.
[INFO] Data limited to 16967752 rows with 3 or more unique products per session.
[INFO:TEST_MODE] Limiting data to the first 10000 sessions...
[INFO:TEST_MODE] Data limited to 89542 rows from the first 10000 sessions.
[INFO] Parsing category hierarchy...
          category sub_category  element
0        computers     notebook  unknown
1        computers      desktop  unknown
2      electronics   smartphone  unkno

In [9]:
# Now try importing
from scripts.preprocessing_scripts.preprocess_graph_with_encoding import preprocess_graph_with_onehot

# Load config
import yaml
with open('../experiments/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

exp_params = config['experiments']['experiment_7']

In [37]:
exp_params['model_params']

{'initial_dimension_dim': 128,
 'num_layers': 5,
 'num_iterations': 1,
 'hidden_units': 256,
 'hidden_dim': 100,
 'dropout_rate': 0.3,
 'optimizer': 'Adam',
 'lr': 0.0005,
 'epochs': 15,
 'batch_size': 32}

In [14]:
import torch
# First, load the datasets
train_path = exp_params['data_params']['output_folder_artifacts'] + "train_dataset.pth"
val_path = exp_params['data_params']['output_folder_artifacts'] + "val_dataset.pth"

print("Loading datasets from:")
print(f"Train: {train_path}")
print(f"Val: {val_path}")

# Load datasets
train_dataset = torch.load(train_path)
val_dataset = torch.load(val_path)

print("\nDataset types:")
print(f"Train dataset type: {type(train_dataset)}")
print(f"Val dataset type: {type(val_dataset)}")


Loading datasets from:
Train: experiments/experiment_7/train_dataset.pth
Val: experiments/experiment_7/val_dataset.pth


/var/folders/tb/spjhzdz13378nkrzqyjljkjm0000gn/T/ipykernel_50064/3928591943.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load(train_path)
/var/


Dataset types:
Train dataset type: <class 'torch.utils.data.dataset.Subset'>
Val dataset type: <class 'torch.utils.data.dataset.Subset'>


In [17]:
from scripts.train_scripts.train_sr_gnn_attn_agg import train_sr_gnn_att_agg_with_onehot


train_sr_gnn_att_agg_with_onehot(
        model_params=exp_params['model_params'],
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        output_folder_artifacts=exp_params['data_params']['output_folder_artifacts'],
        top_k=[20]
)

Using device: cpu
----------------------------------


Training Epoch: 100%|██████████| 250/250 [02:42<00:00,  1.53it/s]


Training: epoch=1/15 | Loss=2136.6003 | R@20=0.0856 | P@20=0.0043 | MRR@20=0.0218


NameError: name 'dataloader' is not defined

In [3]:
from scripts.preprocessing_scripts.preprocess_graph_with_encoding import preprocess_graph_with_onehot

# Load config
with open('experiments/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

exp_params = config['experiments']['experiment_7']

# Preprocess data and create datasets
preprocess_graph_with_onehot(
    input_folder_path=exp_params['data_params']['input_folder_path'],
    output_folder_artifacts=exp_params['data_params']['output_folder_artifacts'],
    preprocessing_params=exp_params['preprocessing']
)

ModuleNotFoundError: No module named 'scripts'